# 第12章 序列的特殊方法

## 12.1 Vector类第1版：与Vector2d类兼容

In [3]:
from array import array
import reprlib
import math


class VectorV1:
    typecode = 'd'

    def __init__(self, components):
        # 将Vector的分量保存在一个数组中
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        # 生成有限长度表示形式
        components = reprlib.repr(self._components) 
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [4]:
VectorV1([3.1, 4.2])

Vector([3.1, 4.2])

In [5]:
VectorV1((3, 4, 5))

Vector([3.0, 4.0, 5.0])

In [6]:
VectorV1(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

## 12.2 Vector类第2版：可切片的序列

- 序列协议：任何类，只要使用标准的签名和语义实现了`__len__`和`__getitem__`方法，就能用在任何预期序列的地方。
- 协议是非正式的，没有强制力的，因此如果知道类的具体使用场景，通常只需要实现协议的一部分。

**改进方案：**
1. 新增`__len__`方法，计算Vector分量的个数。
2. 新增`__getitem__`方法，如果传入的参数是一个区间，则按照区间使用切片生成新的Vector，如果是单个索引，则返回相应的元素。

In [10]:
import operator

class VectorV2:
    typecode = 'd'

    def __init__(self, components):
        # 将Vector的分量保存在一个数组中
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        # 生成有限长度表示形式
        components = reprlib.repr(self._components) 
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            # 如果是一个区间，获取实例的类
            cls = type(self)  
            # 调用类的构造函数，使用切片构建一个新的Vector实例
            return cls(self._components[key])
        # 单个索引
        index = operator.index(key)
        # 返回相应的元素
        return self._components[index]  
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [11]:
v7 = VectorV2(range(7))
v7[-1]

6.0

In [12]:
v7[1:4]

Vector([1.0, 2.0, 3.0])

In [13]:
v7[-1:]

Vector([6.0])

## 12.4 Vector类第3版：动态存取属性

**改进方案：**
1. 新增支持位置模式匹配`__match_args__`。
2. 新增`__getattr__`方法：得到元素的相应位置，如果在范围内，则返回相应的元素。
3. 新增`__setattr__`方法：如果参数是受特性保护的只读属性，则抛出异常，如果是小写字母，则抛出错误警告。

In [17]:
class VectorV3:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]
    
    # 支持位置模式匹配
    __match_args__ = ('x', 'y', 'z', 't')  

    def __getattr__(self, name):
        # 获取Vector类型
        cls = type(self)  
        try:
            # 得到位置
            pos = cls.__match_args__.index(name)  
        except ValueError: 
            pos = -1
        if 0 <= pos < len(self._components):
            # 如果位置处于范围类，则返回对应的分量
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)
        
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            # 处理名称是单个字符的属性
            if name in cls.__match_args__:
                # 受特性保护的只读属性
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        # 调用超类的setattr方法        
        super().__setattr__(name, value)     

In [18]:
v = VectorV3(range(10))
v.x

0.0

In [19]:
v.y, v.z, v.t

(1.0, 2.0, 3.0)

## 12.5 Vector类第4版：哈希和快速等值测试

**改进方案：**
1. 新增`__eq__`方法：先判断两个对象的长度是否相等，再比较其中各个元素是否相等。
2. 新增`__hash__`方法：计算各个分量的哈希值，并使用`xor`运算符聚合所有的哈希值。

In [20]:
class VectorV4:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        # 先判断两个对象的长度是否相等
        # 再比较其中各个元素是否相等
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        # 计算多个分量的哈希值
        hashes = (hash(x) for x in self)
        # v[0]^v[1]^v[2]^...
        # 使用xor运算符聚合所有的哈希值
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    __match_args__ = ('x', 'y', 'z', 't')

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

## 12.6 Vector第5版：格式化

**改进方案：**
1. 新增`angle(self, n)`函数：使用n维球体的公式计算角坐标。
2. 新增`angles`函数：创建生成器表达式，按需计算所有角坐标。
3. 新增`__format__`函数：根据球坐标标识符`h`，显示球面坐标。

In [21]:
import itertools

class VectorV5:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    __match_args__ = ('x', 'y', 'z', 't')

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    def angle(self, n):
        # 使用n维球体的公式计算角坐标
        r = math.hypot(*self[n:])
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):
            # 球坐标标识符h
            fmt_spec = fmt_spec[:-1]
            # 生成器表达式，迭代向量的模和各个角坐标
            coords = itertools.chain([abs(self)], self.angles())  
            # 显示球面坐标
            outer_fmt = '<{}>'  
        else:
            coords = self
            # 显示笛卡尔坐标
            outer_fmt = '({})' 
        components = (format(c, fmt_spec) for c in coords)  
        return outer_fmt.format(', '.join(components))  

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)